In [10]:
# Imports
from subsbml import System, createSubsystem, combineSystems, createNewSubsystem
import numpy as np
import pylab as plt

from bokeh.layouts import row
from bokeh.io import export_png

import warnings
import bokeh.io
import bokeh.plotting

## Cell 1 simple rR1 transport w/ rR1 external reservoir

In [12]:
ss1 = createSubsystem ("BiSwitch_CRN.xml")
ss2 = createSubsystem ("rR1_external_reservoir.xml")

# Create a simple rR1 membrane 
mb1 = createSubsystem("simplemembrane_rR1.xml", membrane = True)

cell_1 = System("cell_1")

cell_1.setInternal([ss1])
cell_1.setExternal([ss2])
cell_1.setMembrane(mb1)

cell_1_model = cell_1.getModel()


#cell_1 = System ("cell_1", ListOfInternalSubsystems = [ss1], 
#                 ListOfExternalSubsystems = [ss2], 
#                 ListOfMembraneSubsystems = [mb1])

##### Set Species Amount #####

cell_1_model.setSpeciesAmount('rna_rR1_e', 200, compartment = 'cell_1_external')
cell_1_model.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('Core2_OFF', 5e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAseH', 50, compartment = 'cell_1_internal')
cell_1_model.setSpeciesAmount('protein_RNAP', 150, compartment = 'cell_1_internal')
#cell_1_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_1_model.writeSBML('cell_1_model.xml')


The subsystem from simplemembrane_rR1.xml has multiple compartments


1

In [13]:
# Calling Names

X_id1 = cell_1_model.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = cell_1_model.getSpeciesByName('Core1_OFF').getId()
X_id3 = cell_1_model.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = cell_1_model.getSpeciesByName('Core2_OFF').getId()
X_id5 = cell_1_model.getSpeciesByName("rna_rR2").getId()
X_id6 = cell_1_model.getSpeciesByName("rna_rR1_i").getId()
X_id7 = cell_1_model.getSpeciesByName("rna_rR1_e").getId()
X_id8 = cell_1_model.getSpeciesByName("complex_Core2_AI").getId()

# Simulate with BioScrape
timepoints = np.linspace(0, 28800, 1000)
timepoints = timepoints/3600
results_1,_ = cell_1_model.simulateWithBioscrape(timepoints)

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
a = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
a.circle(timepoints, results_1[X_id1], legend_label = "Core1 ON" , color = "red")
a.circle(timepoints, results_1[X_id3], legend_label = "Core2 ON", color = "blue")
a.legend.click_policy="hide"
a.legend.location="bottom_right"

b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
#b.circle(timepoints, results_1[X_id5], legend_label = "rR2" , color = "red")
b.circle(timepoints, results_1[X_id6], legend_label = "rR1_i", color = "blue")
b.legend.click_policy="hide"
b.legend.location="center_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, results_1[X_id7], legend_label = "rR1_e", color = "orange")
c.legend.click_policy="hide"
c.legend.location="center_right"

bokeh.io.show(row(a, b, c))
warnings.filterwarnings("ignore")


Loading BokehJS ...

**Notes**

Simple membrane implemented with diffusion of rR1 species.

Bistability is not being shown by either of the cores. RNA production very low. Possible bug in Sub-sbml(?)

In [14]:
d = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
d.circle(timepoints, results_1[X_id8], legend_label = "rR1_i:dA1", color = "orange")
d.circle(timepoints, results_1[X_id9], legend_label = "rR2:dA2", color = "magenta")
d.legend.click_policy="hide"
d.legend.location="bottom_right"

X_id9 = cell_1_model.getSpeciesByName("complex_Core1_AI").getId()
#e = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
#e.circle(timepoints, results_1[X_id9], legend_label = "rR2:dA2", color = "magenta")
#e.legend.click_policy="hide"
#e.legend.location="center_right"

bokeh.io.show(row(d))
warnings.filterwarnings("ignore")

## Cell 2 with transport protein ##

In [16]:
ss1 = createSubsystem ("BiSwitch_CRN.xml")
ss2 = createSubsystem ("rR1_external_reservoir.xml")

# Create a simple rR1 membrane 
mb1 = createSubsystem("RNAexp_membrane3.xml", membrane = True)

cell_2 = System("cell_2")

cell_2.setInternal([ss1])
cell_2.setExternal([ss2])
cell_2.setMembrane(mb1)

cell_2_model = cell_2.getModel()


#cell_1 = System ("cell_1", ListOfInternalSubsystems = [ss1], 
#                 ListOfExternalSubsystems = [ss2], 
#                 ListOfMembraneSubsystems = [mb1])

##### Set Species Amount #####

cell_2_model.setSpeciesAmount('rna_rR1_e', 200, compartment = 'cell_2_external')
cell_2_model.setSpeciesAmount('Core1_OFF', 6e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('Core2_OFF', 5e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_dA2', 6e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('dna_dA1', 6e3, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('protein_RNAseH', 50, compartment = 'cell_2_internal')
cell_2_model.setSpeciesAmount('protein_RNAP', 150, compartment = 'cell_2_internal')

#cell_2_model.getSBMLDocument().getModel().getCompartment(1).setSize(1e-4) 

cell_2_model.writeSBML('cell_2_model.xml')

The subsystem from RNAexp_membrane3.xml has multiple compartments


1

In [21]:
# Calling Names

X_id1 = cell_2_model.getSpeciesByName('complex_Core1_ON').getId()
X_id2 = cell_2_model.getSpeciesByName('Core1_OFF').getId()
X_id3 = cell_2_model.getSpeciesByName('complex_Core2_ON').getId()
X_id4 = cell_2_model.getSpeciesByName('Core2_OFF').getId()
X_id5 = cell_2_model.getSpeciesByName("rna_rR2").getId()
X_id6 = cell_2_model.getSpeciesByName("rna_rR1_i").getId()
X_id7 = cell_2_model.getSpeciesByName("rna_rR1_e").getId()
X_id8 = cell_2_model.getSpeciesByName("complex_Core2_AI").getId()

# Simulate with BioScrape
timepoints = np.linspace(0, 28800, 1000)
timepoints = timepoints/3600
results_2,_ = cell_2_model.simulateWithBioscrape(timepoints)

# For label convenience
x = 'Time (hours)'
y = 'Concentration (uM)'

bokeh.io.output_notebook()
b = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
#b.circle(timepoints, results_1[X_id5], legend_label = "rR2" , color = "red")
b.circle(timepoints, results_2[X_id6], legend_label = "rR1_i", color = "blue")
b.legend.click_policy="hide"
b.legend.location="center_right"

c = bokeh.plotting.figure(plot_width=400, plot_height=300, x_axis_label = x, y_axis_label=y)
c.circle(timepoints, results_2[X_id7], legend_label = "rR1_e", color = "orange")
c.legend.click_policy="hide"
c.legend.location="center_right"

bokeh.io.show(row(b, c))

Loading BokehJS ...